In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# 사전 훈련된 BERT 모델과 토크나이저 불러오기
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 분류 모델 생성
classification_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 숫자 예측 예제 (숫자를 텍스트로 변환하여 입력)
text = "The temperature today is 25 degrees Celsius."
result = classification_model(text)

# 결과 출력
print(f"Predicted Sentiment (Class Label): {result[0]['label']}")

2023-09-24 13:54:58.250538: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Sentiment (Class Label): LABEL_0


In [4]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertConfig

# BERT 모델 아키텍처를 회귀 모델로 수정
class BertForRegression(nn.Module):
    def __init__(self, config):
        super(BertForRegression, self).__init__()
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, 1)  # 회귀를 위한 선형 레이어

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] 토큰의 출력을 사용
        pooled_output = self.dropout(pooled_output)
        regression_output = self.regressor(pooled_output)
        return regression_output

# 사전 훈련된 BERT 모델 및 토크나이저 불러오기
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
model = BertForRegression(config)

# 숫자 데이터를 입력으로 변환
text = "cont_cap year month day dayofweek dayofyear eid_code_num eid_num office_num sl_cnts ..."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# 숫자 데이터 예측
output = model(**inputs)
predicted_value = output.item()

# 결과 출력
print(f"Predicted Value: {predicted_value}")

TypeError: BertForRegression.forward() got an unexpected keyword argument 'token_type_ids'